In [36]:
from pathlib import Path
import json
import pandas as pd
import gc # сборщик мусора
import pickle
from imputer import FeatureImputer

In [37]:
gc.collect()

40

In [38]:
# paths
RAW_DATA_PATH = Path('./data_raw')
TEST_RAW_PATH = RAW_DATA_PATH / 'test.csv'
SCALERS_PATH = Path('./scalers')

# версия моделей
MODEL_TAKE_NO = '01_bal_smote'
MODEL_PATH = Path('./models/' + MODEL_TAKE_NO)

# версия предобработки
IMPUTER_METHOD = '01_nan'

In [39]:
test = pd.read_csv(TEST_RAW_PATH)
test.head(2)

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score
0,Rent,NaN,4 years,0.0,9.0,12.5,220968.0,0.0,70.0,0.0,debt consolidation,Short Term,162470.0,105906.0,6813.0,NaN
1,Rent,231838.0,1 year,0.0,6.0,32.7,55946.0,0.0,8.0,0.0,educational expenses,Short Term,78298.0,46037.0,2318.0,699.0


In [40]:
# загрузить словарь признаков
with open("feature_dict.json", "r") as f:
    feature_dict = json.load(f)
# заполнить признаки по словарю
NUM_FEATURE_NAMES = feature_dict['NUM_FEATURE_NAMES']
CAT_FEATURE_NAMES = feature_dict['CAT_FEATURE_NAMES']
TARGET_NAME = feature_dict['TARGET_NAME']

In [41]:
def get_selected_features():
    drop_feature_list = ['Tax Liens']
    selected_features = NUM_FEATURE_NAMES
    for feature_name in drop_feature_list:
        if feature_name in selected_features:
            selected_features.remove(feature_name)
    return selected_features

In [42]:
preds_final = pd.DataFrame()
preds_final['Id'] = test.index

In [43]:
test = test[NUM_FEATURE_NAMES]
# предобработка
imputer = FeatureImputer(test, IMPUTER_METHOD)
test = imputer.transform()

In [44]:
# стандартизация
# scaler_file = SCALERS_PATH / 'st_scaler.pkl'
# scaler = pickle.load(open(scaler_file, 'rb'))
# test = scaler.transform(test)
# test = pd.DataFrame(test,columns = NUM_FEATURE_NAMES)

In [45]:
test.tail(10)

,Annual Income,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score
2490,1494464.0,0.0,5.0,15.7,553234.0,0.0,30.0,0.0,179784.0,177745.0,3437.0,751.0
2491,916199.0,0.0,16.0,12.4,316382.0,0.0,73.0,0.0,442024.0,88521.0,2313.0,738.0
2492,750500.0,0.0,5.0,6.3,162998.0,0.0,30.0,0.0,173800.0,110238.0,5741.0,690.0
2493,2038282.0,0.0,15.0,12.6,847000.0,0.0,30.0,0.0,432256.0,88521.0,27177.0,692.0
2494,907839.0,0.0,14.0,11.1,438526.0,0.0,30.0,0.0,368786.0,88255.0,18838.0,705.0
2495,1020053.0,0.0,14.0,29.1,559152.0,1.0,68.0,1.0,99999999.0,162735.0,15046.0,745.0
2496,1178475.0,0.0,15.0,17.0,1737780.0,0.0,77.0,0.0,468512.0,1439269.0,32996.0,731.0
2497,1171806.0,0.0,48.0,12.8,1706430.0,0.0,30.0,0.0,430496.0,676438.0,36912.0,695.0
2498,723520.0,0.0,14.0,28.8,945780.0,0.0,30.0,0.0,257774.0,391248.0,13506.0,744.0
2499,1694439.0,0.0,12.0,18.4,1199748.0,1.0,72.0,0.0,763004.0,559531.0,23440.0,682.0


In [46]:
SELECTED_FEATURES = get_selected_features()
test = test[SELECTED_FEATURES]

In [47]:
print(SELECTED_FEATURES)

['Annual Income', 'Number of Open Accounts', 'Years of Credit History', 'Maximum Open Credit', 'Number of Credit Problems', 'Months since last delinquent', 'Bankruptcies', 'Current Loan Amount', 'Current Credit Balance', 'Monthly Debt', 'Credit Score']


In [48]:
model_file = MODEL_PATH / ('model_xgb_num01.pkl')
xgb_model = pickle.load(open(model_file, 'rb'))
y_final = xgb_model.predict(test)

In [49]:
preds_final[TARGET_NAME] = y_final
preds_final.to_csv('./submission.csv', index=False)